Puzzle Solver: Cryptarithm Puzzle Solver Using DFS Algorithm

In [12]:
#  class to solve cryptarithmetic puzzles using  DFS algorithm.
class PuzzleSolverDFS:
    # Constructor 
    def __init__(self, puzzle):
        # Storing the puzzle.
        self.puzzle = puzzle
        # Extracting unique letters from the puzzle.
        self.unique_letters = set(puzzle.replace(' ', ''))
        # Initialize mapping 
        # for each letter in the puzzle  to None.
        self.mapping = {letter: None for letter in self.unique_letters}

    # Method/ function to solve the puzzle.
    def solve(self):
        # Spliting the puzzle into left and right operands 
        left_operand, right_operand = self.puzzle.replace(' ', '').split('=')[0].split('+')
        result = self.puzzle.replace(' ', '').split('=')[1]

        # checking if the current mapping is valid.

        def is_valid(mapping):

            # checking if all letters have been assigned a digit.
            if all(mapping[letter] is not None for letter in self.unique_letters):
                # confirm that leading zeros are not assigned to any operand or result in the puzzle.
                
                if all(mapping[op[0]] != '0' for op in (left_operand, right_operand, result)):
                    return True
            return False

       # exploring possible solutions.
         # Depth-First Search 
        def dfs(mapping):
            # If the current mapping is correct
            if is_valid(mapping):
               
                if self.evaluate(left_operand, mapping) + self.evaluate(right_operand, mapping) == self.evaluate(result, mapping):
                    return mapping
                return None

            try:
                # finding the next unassigned letter.
                letter = next(letter for letter in self.unique_letters if mapping[letter] is None)
            except StopIteration:
                return None

            # assigning each digit (0-9) 
            # to the unassigned letter in the puzzle
            for digit in range(10):
                if str(digit) in mapping.values():
                    continue

                mapping[letter] = str(digit)
                solution = dfs(mapping.copy())
                if solution:
                    return solution

            # Backtracking if no solution is found.
            mapping[letter] = None
            return None

        # Starting DFS with the initial mapping(make sure its valid).
        return dfs(self.mapping)

    # evaluation of an expression using the current mapping.
    def evaluate(self, expression, mapping):
        return int(''.join(mapping[letter] for letter in expression))



class SolutionPrinter:
    # Static method to print solution.
    @staticmethod
    def print_solution(puzzle, solution):
        if solution:
            print("Solution found:")
            puzzle_no_space = puzzle.replace(" ", "")
            left_operand, right_operand = puzzle_no_space.split('=')[0].split('+')
            result = puzzle_no_space.split('=')[1]
            for word, value in solution.items():
                print(f"{word}: {value}")
            print(left_operand + ": " + ''.join(solution[letter] for letter in left_operand))
            print("+ " + right_operand + ": " + ''.join(solution[letter] for letter in right_operand))
            print("= " + result + ": " + ''.join(solution[letter] for letter in result))
        else:
            print("No solution found for the puzzle.")


class CryptarithmeticSolver:
    # Constructor 
    def __init__(self, puzzle):
         # initialize the puzzle.
        self.puzzle = puzzle

  
    def solve_and_print(self):
        # instance of PuzzleSolverDFS to solve the puzzle.
        solver = PuzzleSolverDFS(self.puzzle)
        solution = solver.solve()
        SolutionPrinter.print_solution(self.puzzle, solution)


# Example of list of puzzles 
puzzles = [
     
     "BREAD + WINE = DINER",
     "EARTH + MARS = PLANETS" # this one does not give solution
]

for puzzle in puzzles:
    print("\nPuzzle:", puzzle)
    solver = CryptarithmeticSolver(puzzle)
    solver.solve_and_print()



Puzzle: BREAD + WINE = DINER
Solution found:
A: 4
D: 3
+: 0
R: 8
W: 7
B: 2
N: 1
=: 9
I: 6
E: 5
BREAD: 28543
+ WINE: 7615
= DINER: 36158

Puzzle: EARTH + MARS = PLANETS
No solution found for the puzzle.


Puzzle Solver: Cryptarithm Puzzle Solver Using A* Algorithm

In [11]:
# used for generating permutations.
import itertools  
# used for priority queue 
import heapq  

# class to represent cryptarithm puzzle 
class Cryptarithm: 

     # Constructor 
    def __init__(self, expression):  
        self.expression = expression  
        self.initial_state = self._initialize_state()  
        self.goal_state = self._extract_goal_state()  

    #initializing the state with unique letters.
    def _initialize_state(self):  
        expression_set = set(self.expression.replace(' ', '').replace('+', '').replace('=', ''))
        # Initializing each letter with a placeholder value.
        return {letter: -1 for letter in expression_set}  

    #  extract the goal state from the expression.
    def _extract_goal_state(self):  #
        return self.expression.split('=')[-1].strip()

      # Getter method 
    def get_initial_state(self): 
        return self.initial_state

     # Getter method to get the goal state.
    def get_goal_state(self): 
        return self.goal_state
    

# class to show nodes in the search space.
class PuzzleNode:  

 # Constructor 
    def __init__(self, state, parent=None): 
        self.state = state  
        self.parent = parent  
        # Initializing the heuristic cost of the node
         # to zero.
        self.cost = 0 

    def __lt__(self, other):  
        #  less than comparison for nodes based on their costs
        return self.cost < other.cost

class CryptarithmSolver: 
    # Constructor
    def __init__(self, expression): 
        self.expression = expression  

    def solve(self):  
        initial_state = self.expression.get_initial_state()  
        goal_state = self.expression.get_goal_state()  

          # heuristic funct to estimate the cost to reach the goal state.
        heuristic = lambda state: sum(10**i * (9 - state[letter]) for i, letter in enumerate(goal_state[::-1]))
      
        # Generating all possible permutations of digits.
        permutations = itertools.permutations(range(10))  
         # Initialize the priority queue 
        frontier = [] 
        # Iterate through each permutation.
        for perm in permutations:  
            mapping = {letter: digit for letter, digit in zip(initial_state.keys(), perm)}
            # Create a mapping from letters to digits based on the permutation.
                  # Skip permutations with leading zeros
            if mapping[goal_state[0]] == 0:
                continue 
            
            node = PuzzleNode(mapping)  # Createing a puzzle node with the mapping.
            node.cost = heuristic(mapping)  # than calculating the heuristic cost of the node.
            heapq.heappush(frontier, node)  #  snd then adding the node to the priority queue.

        # a set to keep track of explored states.
        explored = set()  

          # Continue until the frontier is empty , right now it is not empty
        while frontier: 
             # Get/pop the node with the lowest cost from the priority queue.
            current_node = heapq.heappop(frontier)  

            if self.evaluate_expression(current_node.state): 
                return current_node.state  

           # Adding the current state to the set of explored states.
            explored.add(tuple(current_node.state.values()))  

            for next_state in self.get_next_states(current_node.state):  
                 # Checking if the next state has not been explored.
                if tuple(next_state.values()) not in explored: 
                      # Creating a new node which is the next node for the next state.
                    next_node = PuzzleNode(next_state, parent=current_node)  
                    next_node.cost = heuristic(next_state)  
                    # Adding the next node to the priority queue.
                    heapq.heappush(frontier, next_node)  

        return None  # Return None if no solution is found.
      
        #state generation using given state
    def get_next_states(self, state):  
        next_states = []  
        # digits from 0 to 9.
        for digit in range(10): 
             # Checking if the digit is not already used. 
            if digit not in state.values():  
                  # Iterate through letters in the state.
                for letter, value in state.items(): 
                      # Checking if the letter has not been assigned a digit yet or is assigned -1
                    if value == -1:  
                        if digit == 0 and letter in state.keys() - {'='}:  # Skip leading zeros 
                            continue
                        # Create a copy of the current state and also a assign the digit to the letter in the next state.
                        next_state = state.copy()  
                        next_state[letter] = digit  
                        # Add the next state to the list.
                        next_states.append(next_state)  
                        break
        return next_states  # Return the list of next states.

    def evaluate_expression(self, mapping):  
        # Spliting the expression into operands and result.
        operands, result = self.expression.expression.split('=')  
        # Remove  whitespace.
        result = result.strip()  
         # Split the operands by the plus sign.
        operands = operands.split('+') 

        total = 0  # a variable to store the total value of the operands.
        for operand in operands:  
            # Calculating the value of the operand based on the mapping.
            operand_value = int(''.join(str(mapping[letter]) for letter in operand.strip()))
            
            total += operand_value  # Adding the operand value to the total.
        
        result_value = int(''.join(str(mapping[letter]) for letter in result.strip()))
        
        return total == result_value  
           # Return True if the total matches the result, otherwise False.

class AStarCryptarithmSolver: 
    def __init__(self, expression): 
        self.expression = expression  
  
   # solve the cryptarithm problem using A* algorithm.
    def solve(self):  
        solver = CryptarithmSolver(self.expression)
        return solver.solve()  

def print_solution(solution, expression):  
    if solution:  
        print("Solution found:")  
        for letter, digit in solution.items(): 
            print(f"{letter}: {digit}")  
        
        # Split the expression into operands and result and than Split the operands by the plus sign.
        operands, result = expression.split('=')  
        operands = operands.split('+')  
        for operand in operands:  
            operand_value = int(''.join(str(solution[letter]) for letter in operand.strip()))

            # Calculating the value of the operand based on the solution.
            print(f"{operand.strip()}: {operand_value}") 
        
        result_value = int(''.join(str(solution[letter]) for letter in result.strip()))

        # Calculating the value of the result based on the solution.
        print(f"{result.strip()}: {result_value}")  
    else:  
        print("No solution found.")  

# Example 
expression_list = [  
      "SEND + MORE = MONEY",
      "STAR + STAR = NOVA",
      "SODA + POP = FIZZ"
]

for expression in expression_list:  
    cryptarithm = Cryptarithm(expression)  
    solver = AStarCryptarithmSolver(cryptarithm)  
    solution = solver.solve()  
    print_solution(solution, expression)  


Solution found:
D: 7
R: 8
O: 0
E: 5
N: 6
M: 1
S: 9
Y: 2
SEND: 9567
MORE: 1085
MONEY: 10652
Solution found:
A: 6
R: 3
O: 7
T: 8
N: 9
S: 4
V: 2
STAR: 4863
STAR: 4863
NOVA: 9726
Solution found:
A: 5
D: 3
O: 7
P: 6
I: 4
Z: 1
F: 9
S: 8
SODA: 8735
POP: 676
FIZZ: 9411
